In [2]:
import gdspy
from design_coaxmons import *
from loops_and_JJs_coaxmons import*
from coaxmons import *
from airbridges import *

In [3]:
class Sample:
    
    def __init__(self, number):
        self.name = 'sample' + str(number)
        self.cell = gdspy.Cell(self.name)
        self.restricted_area = []
        
    def Generate_Pads(self, a, b, number, d_arr, d1_arr, d2_arr, layer, coords = []):
        self.Pads = Pads(a, b, number, d_arr, d1_arr, d2_arr, layer, coords)
        self.Pads.Generate_Ground()
        #self.Pads_cell = gdspy.Cell('Pads2')
        #self.Pads_cell.add(self.Pads.Generate_Ground())
        #self.cell.add(gdspy.CellReference(self.Pads_cell, (0, 0)))
        
    def Generate_Individual_Pads(self, a, b, number, d_arr, d1_arr, d2_arr, layer, coords = []):
        self.Pads = Pads(a, b, number, d_arr, d1_arr, d2_arr, layer, coords)
        self.Pads_cell = gdspy.Cell('Pads')
        self.Pads_cell.add(self.Pads.Generate_Ground())
        self.cell.add(gdspy.CellReference(self.Pads_cell, (0, 0)))
    
    #def Generate_TL(self, start_pad, finish_pad, d, d1, d2, layer):
    #    TL = Transmission_Line(self.Pads, start_pad, finish_pad, d, d1, d2, layer)
    #    self.tl = TL
    #    self.tl_cell = gdspy.Cell('TL')
    #    self.tl_cell.add(TL.Generate(coords = [], rots = [])) 
        #self.cell.add([gdspy.CellReference(self.tl_cell, (0, 0))])
        
    def Generate_TL(self, start_pad, finish_pad, d, d1, d2, layer):
        TL = Transmission_Line(self.Pads, start_pad, finish_pad, d, d1, d2, layer)
        self.tl = TL
        self.tl_cell1 = gdspy.Cell('TL1')
        self.tl_cell2 = gdspy.Cell('TL2')
        self.tl_cell3 = gdspy.Cell('TL3')
        self.tl_cell1.add(TL.GenerateTL(d, coords = [], rots = [])) 
        self.tl_cell2.add(TL.GenerateTL(d1, coords = [], rots = [])) 
        self.tl_cell3.add(TL.GenerateTL(d2, coords = [], rots = [])) 
    
    def Generate_Resonators(self, xs, top_TL, middle_TL, R2s, outer_grounds, R4s, freqs, DE, TL_ground, d, d1, d2, modes):
        self.resonators = []
        self.resonators1_cell = gdspy.Cell('resonators1')
        self.resonators2_cell = gdspy.Cell('resonators2')
        self.resonators3_cell = gdspy.Cell('resonators3')
        
        for i, (x, R2, outer_ground, R4, freq, mode) in enumerate(zip(xs, R2s, outer_grounds, R4s, freqs, modes)):
            self.resonators.append(Resonator(x, top_TL , R2, outer_ground, R4, freq, d, d1, d2))
            self.resonators1_cell.add(self.resonators[i].Generate(DE, TL_ground, d, middle_TL, mode))
            self.resonators2_cell.add(self.resonators[i].Generate(DE, TL_ground, d1, middle_TL, mode))
            self.resonators3_cell.add(self.resonators[i].Generate(DE, TL_ground, d2, middle_TL, mode))        
    
    def Generate_Coaxmons(self, r1s, r2s, r3s, R4s, outer_grounds, arcs, modes, middle_TL):
        self.coaxmons = []
        self.coaxmons_cell = gdspy.Cell('coaxmons1')
        for i, (r1, r2, r3, R4, outer_ground, arc, mode) in enumerate(zip(r1s, r2s, r3s, R4s, outer_grounds, arcs, modes)):
            if mode == 'up':
                sample.restricted_area.append(gdspy.Round((sample.resonators[i].center.x, sample.resonators[i].center.y), outer_ground, 0, initial_angle=0, final_angle=2*np.pi))
            else:
                sample.restricted_area.append(gdspy.Round((sample.resonators[i].center.x, 2*middle_TL-sample.resonators[i].center.y), outer_ground, 0, initial_angle=0, final_angle=2*np.pi))
            self.coaxmons.append(Coaxmon(sample.resonators[i].center, r1, r2, r3, R4, outer_ground, arc))
            self.coaxmons_cell.add(self.coaxmons[i].Generate(mode, middle_TL))
            #self.cell.add(self.coaxmons[i].Generate(mode, top_TL, middle_TL, DE, TL_ground))
            
    def Generate_Loops(self, rightc, modes, d, d1, d2, d_tl, d1_tl, d2_tl, R, coords_arr, layer):#
        self.loops = []
        self.loop_cells = []
        for i, (right, mode, coords) in enumerate(zip(rightc, modes, coords_arr)):#
            self.loops.append(Loop(right.x, right.y, d, d1, d2, d_tl, d1_tl, d2_tl, mode, 
                                   self.Pads.reference_points[i + 2], layer))
            self.loop_cells.append(gdspy.Cell('loop' + str(i)))
            self.loop_cells[i].add(self.loops[i].Generate(coords, R, mode))

            self.cell.add(gdspy.CellReference(self.loop_cells[i], (0,0)))

            
    def Merge(self, DE):
        area_loops = self.loops[0].restricted_area
        for i in range(1,len(self.loops)):
            area_loops = gdspy.boolean(self.loops[i].restricted_area, area_loops, 'or')
        cut_coaxmon = gdspy.Cell('cut_coaxmon')
        cut_coaxmon.add(gdspy.boolean(gdspy.CellReference(self.coaxmons_cell, (0,0)), area_loops, 'not'))
        
        all_3 = gdspy.Cell('all_3')
        all_3.add(gdspy.boolean(gdspy.CellReference(cut_coaxmon, (0,0)), (gdspy.boolean(gdspy.CellReference(self.tl_cell3, (0,0)), gdspy.CellReference(self.resonators3_cell, (0,0)), 'or')), 'or'))
        #restricted area!
        sample.restricted_area.append(gdspy.CellReference(all_3, (0,0)))
        all_2 = gdspy.Cell('all_2')
        all_2.add(gdspy.boolean(gdspy.CellReference(self.tl_cell2, (0,0)), gdspy.CellReference(self.resonators2_cell, (0,0)), 'or'))
        all_1 = gdspy.Cell('all_1')
        all_1.add(gdspy.boolean(gdspy.CellReference(self.tl_cell1, (0,0)), gdspy.CellReference(self.resonators1_cell, (0,0)), 'or'))
        cut = gdspy.Cell('cut')
        cut.add(gdspy.boolean(gdspy.CellReference(all_3, (0,0)), gdspy.CellReference(all_2, (0,0)), 'not'))
        result = gdspy.Cell('result')
        result.add(gdspy.boolean(gdspy.CellReference(all_1, (0,0)), gdspy.CellReference(cut, (0,0)), 'or', layer = 10))
        self.cell.add(gdspy.CellReference(result, (0, 0)))
        
    def Generate_JJ(self, corners, r1, r2, r3, r4, r5, r6, layer, modes, mirror_x, mirror_y):
        self.jj = []
        self.jj_cells = []
        self.jj_cell = gdspy.Cell('JJs')
        for i, (corner, a, b, c, d, w1, w2, mode) in enumerate(zip(corners, r1, r2, r3, r4, r5, r6, modes)):
            self.jj.append(JJ(a, b, c, d, w1, w2, layer))
            #self.jj_cells.append(gdspy.Cell('jj' + str(i)))
            #self.jj_cells[i].add(self.jj[i].Generate(corner.x, corner.y, mode, mirror_x, mirror_y))
            self.jj_cell.add(self.jj[i].Generate(corner.x, corner.y, mode, mirror_x, mirror_y))
            #self.cell.add(gdspy.CellReference(self.jj_cells[i], (0,0)))#(corner.x, corner.y)))
            self.cell.add(gdspy.CellReference(self.jj_cell, (0,0)))#(corner.x, corner.y)))

    def Generate_Airbridge(self, widths, lengths, padsizes, coords, layers, modes, mirror_x, mirror_y):
        self.ab = []
        self.ab_contacts_cell = gdspy.Cell('ABs_contacts')
        self.ab_bridge_cell = gdspy.Cell('ABs_bridge')
        for i, (width, length, padsize, coord, mode) in enumerate(zip(widths, lengths, padsizes, coords, modes)):
            self.ab.append(Airbridge(width, length, padsize, coord))
            self.ab_contacts_cell.add(self.ab[i].Generate_contacts(layers[0], mode, mirror_x, mirror_y))
            self.ab_bridge_cell.add(self.ab[i].Generate_bridge(layers[1], mode, mirror_x, mirror_y))
            #self.cell.add(gdspy.CellReference(self.ab_contacts_cell, (0,0)))
            #self.cell.add(gdspy.CellReference(self.ab_bridge_cell, (0,0)))
            
            
    def Generate_Airbridge_TL(self, x, y, width, length, padsize, layers):
        self.ab_TL = []
        for i in range(len(x)):
            self.ab_TL.append(Airbridge_TL(x[i], y, width, length, padsize))
            self.ab_contacts_cell.add(self.ab_TL[i].Generate_contacts(layers[0]))
            self.ab_bridge_cell.add(self.ab_TL[i].Generate_bridge(layers[1]))
            self.cell.add(gdspy.CellReference(self.ab_contacts_cell, (0,0)))
            self.cell.add(gdspy.CellReference(self.ab_bridge_cell, (0,0)))
            
    def Generate_Grid(self, a, b, l1, l2):
        self.grid_cell = gdspy.Cell('Grid')
        res1 = gdspy.Rectangle((400, 400), (404, b - 400), layer = l1) 
        x = 447
        while x < a - 400:
            r1 = gdspy.Rectangle((x, 400), (x + 2, b - 400), layer = l1) 
            x += 47
            res1 = gdspy.boolean(res1, r1, 'or')
            #self.grid_cell.add(r1)
        y = 400
        res2 = gdspy.Rectangle((400, 400), (400, 404), layer = l2) 
        while y < b - 400:
            r1 = gdspy.Rectangle((400, y), (a - 400, y + 2), layer = l2) 
            y += 47
            res2 = gdspy.boolean(res2, r1, 'or')
            #self.grid_cell.add(r1)
        res2 = gdspy.boolean(res2, res1, 'not')
        for area in self.Pads.restricted_area:
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        for area in sample.restricted_area:
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        #res = gdspy.boolean(res, self.Pads_cell.get_polygonsets(), 'not')
        for i in range(len(self.loops)):
            area = self.loops[i].restricted_area
            res1 = gdspy.boolean(res1, area, 'not', layer = l1)
            res2 = gdspy.boolean(res2, area, 'not', layer = l2)
        self.grid_h = res1
        self.grid_v = res2
        self.grid_cell.add(res1)
        self.grid_cell.add(res2)
        self.cell.add(gdspy.CellReference(self.grid_cell, (0, 0)))
    

    
    

In [4]:
xxx = np.linspace(900, 8500, 10)
xx = np.repeat(xxx, 2)
x = xx[0:20]
TL_core = 8.5
TL_vacuum = 5
TL_ground = 40 #21
DE = 8.7
d = 16
d1 = 26
d2 = 3*d1
r1 = np.repeat(0.5, len(x))
r2 = np.repeat(0.8, len(x))
r3 = np.repeat(0.91371, len(x))
R4 = np.repeat(207.93869, len(x))
arc_length = np.repeat(1/13.5, len(x))
R3 = r3*R4
R2 = r2*R4
R1 = r1*R4
outer_ground = 1.8*R4
freq = 1e9*np.array([7.121052632, 6.805263158, 7.278947368, 7.226315789, 7.173684211, 7.7, 7.331578947, 7.594736842, 7.489473684, 7.068421053,
        7.384210526, 6.857894737, 6.752631579, 6.7, 6.963157895, 7.015789474, 7.436842105, 7.542105263, 6.910526316, 7.647368421])

orientations = np.tile(['up', 'down'], len(x))
orientations_inverse = np.tile(['down', 'up'], [len(x)])
JJ_BL = 1e-3*np.array([315.9261459, 459.6794258, 398.1994378, 507.9251913, 282.125235, 428.4584492, 263.6504233, 292.6673965, 443.9612296,
         272.4220288, 328.4888088, 491.6791513, 355.1507346, 341.5832779, 303.9612346, 255.9062328, 383.5021155, 369.1390297,
         475.5916799, 413.1953319]) #thickness of rectangle bottom left

JJ_BR = 1e-3*np.array([210.5263158, 394.7368421, 321.0526316, 450, 155.2631579, 357.8947368, 118.4210526, 173.6842105, 376.3157895,
         136.8421053, 228.9473684, 431.5789474, 265.7894737, 247.3684211, 192.1052632, 100, 302.6315789, 284.2105263,
         413.1578947, 339.4736842]) #thickness of rectangle bottom right



JJ_TL = JJ_BL #thickness of rectangle top top
JJ_TR = JJ_BR #thickness of rectangle bottom right
JJ_L = np.repeat(17, len(x)) # length of JJ (h)
JJ_Tw = np.repeat(13, len(x)) #width of upper part of JJ
JJ_Bw = np.repeat(7, len(x)) #width of lower part of JJ

d_loop = 4
d1_loop = 14
d2_loop = 22

40


In [4]:
sample = Sample(1)
sample.Generate_Pads(10000, 5000, #width, depth
                     2, #number of Pads
                     [TL_core, TL_core], #d
                     [TL_core + 2*TL_vacuum, TL_core + 2*TL_vacuum], #d1
                     [TL_core + 2*(TL_ground+TL_vacuum), TL_core + 2*(TL_ground+TL_vacuum)], #d2
                     10, #layer
                     [coordinates(600, 3740), coordinates(9400, 3740)]) #reference coordinates for Pads

sample.Generate_TL(0, 1, TL_core, TL_core + 2*TL_vacuum, TL_core + 2*(TL_vacuum + TL_ground), 0)
top_TL = sample.tl.middle.y + TL_core/2 + TL_vacuum + TL_ground
middle_TL = sample.tl.middle.y
    
sample.Generate_Resonators(x, #x
                          top_TL,
                          middle_TL,
                          R2,
                          outer_ground,
                          R4,
                          freq,
                          DE,
                          TL_ground,
                          d,
                          d1,
                          d2,
                          orientations)

sample.Generate_Coaxmons(r1, #r1
                         r2,  #r2
                         r3, #r3
                         R4,  #R4
                         outer_ground,
                         arc_length, #arc length
                         orientations, middle_TL)
JJ_coordinates = []
for i in range(len(x)):
    JJ_coordinates.append(sample.coaxmons[i].JJ_coordinates)

sample.Generate_JJ(JJ_coordinates,
                   JJ_BL, #thickness of rectangle bottom left
                   JJ_BR, #thickness of rectangle bottom right
                   JJ_TL, #thickness of rectangle top left
                       JJ_TR, #thickness of rectangle top right
                       #JJ_L, # length of JJ (h)
                       JJ_Tw, #width of top part of JJ
                       JJ_Bw, #width of bottom part of JJ
                       1, #layer
                       orientations,
                       sample.coaxmons[0].center.x,
                       middle_TL)

Pad_coordinates = []
Pad_coordinates.append(coordinates(600, 3740))
Pad_coordinates.append(coordinates(9400, 3740))
Pad_coordinates_x = np.linspace(1100,8900,12)
for i in range(ceil(len(x)/2)):
    if i < 5:
        Pad_coordinates.append(coordinates(Pad_coordinates_x[i], 4300))
        Pad_coordinates.append(coordinates(Pad_coordinates_x[i], 700))
    else:
        Pad_coordinates.append(coordinates(Pad_coordinates_x[i+2], 4300))
        Pad_coordinates.append(coordinates(Pad_coordinates_x[i+2], 700))

sample.Generate_Individual_Pads(10000, 5000, #width, depth
                     22, #number of Pads
                     [TL_core, TL_core, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10], #d
                     [TL_core + 2*TL_vacuum, TL_core + 2*TL_vacuum, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22], #d1
                     [TL_core + 2*(TL_ground+TL_vacuum), TL_core + 2*(TL_ground+TL_vacuum), 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64, 64], #d2
                     10, #layer
                     Pad_coordinates) #reference coordinates for Pads

AB_widths = np.tile(15, len(x)*10)
AB_lengths = np.tile(100, len(x)*10)
AB_padsize = 25
AB_padsizes = np.tile(AB_padsize, 10*len(x))
orientations_double = np.repeat(orientations,10)
AB_coordinates = []
for i in range(len(x)):
    for j in range(5):
        AB_coordinates.append(coordinates(sample.coaxmons[i].AB1_coordinates.x, (sample.coaxmons[i].AB1_coordinates.y + j*1.2*AB_padsize)))
        AB_coordinates.append(coordinates(sample.coaxmons[i].AB2_coordinates.x, (sample.coaxmons[i].AB2_coordinates.y + 10 + j*1.2*AB_padsize)))

sample.Generate_Airbridge(AB_widths, AB_lengths, AB_padsizes, AB_coordinates,
                         [2,3], orientations_double,
                       sample.coaxmons[0].center.x,
                       middle_TL)

x_AB_TL = xxx - 100
AB_TL_width = 15
AB_TL_length = 60
AB_TL_padsize = 20
sample.Generate_Airbridge_TL(x_AB_TL, middle_TL, AB_TL_width, AB_TL_length, AB_TL_padsize, [2,3])

rot = [[rotations(np.pi, np.pi/2), rotations(3*np.pi/2, 2*np.pi)], [rotations(1*np.pi, (1+1/2)*np.pi), rotations(1*np.pi/2, 0)]]
rots = np.tile(rot, len(xx)-1)

sample.Generate_Loops([coordinates(sample.jj[0].ref_x - 6, sample.jj[0].ref_y), 
                       coordinates(sample.jj[1].ref_x + 6, sample.jj[1].ref_y),
                       coordinates(sample.jj[2].ref_x - 6, sample.jj[2].ref_y), 
                       coordinates(sample.jj[3].ref_x + 6, sample.jj[3].ref_y),
                       coordinates(sample.jj[4].ref_x - 6, sample.jj[4].ref_y), 
                       coordinates(sample.jj[5].ref_x + 6, sample.jj[5].ref_y),
                       coordinates(sample.jj[6].ref_x - 6, sample.jj[6].ref_y), 
                       coordinates(sample.jj[7].ref_x + 6, sample.jj[7].ref_y),
                       coordinates(sample.jj[8].ref_x - 6, sample.jj[8].ref_y), 
                       coordinates(sample.jj[9].ref_x + 6, sample.jj[9].ref_y),
                       coordinates(sample.jj[10].ref_x - 6, sample.jj[10].ref_y), 
                       coordinates(sample.jj[11].ref_x + 6, sample.jj[11].ref_y), 
                       coordinates(sample.jj[12].ref_x - 6, sample.jj[12].ref_y), 
                       coordinates(sample.jj[13].ref_x + 6, sample.jj[13].ref_y), 
                       coordinates(sample.jj[14].ref_x - 6, sample.jj[14].ref_y), 
                       coordinates(sample.jj[15].ref_x + 6, sample.jj[15].ref_y),
                       coordinates(sample.jj[16].ref_x - 6, sample.jj[16].ref_y), 
                       coordinates(sample.jj[17].ref_x + 6, sample.jj[17].ref_y),
                       coordinates(sample.jj[18].ref_x - 6, sample.jj[18].ref_y), 
                       coordinates(sample.jj[19].ref_x + 6, sample.jj[19].ref_y)], 
                      orientations_inverse, d_loop, d1_loop, d2_loop, 10, 22, 64, R4[0]-R1[0] - sample.jj[0].height -0.6,
                      [[coordinates(2087, 4200), coordinates(2527, 4264)],
                       [coordinates(2526, 850), coordinates(3354, 1083)],
                       [coordinates(2087, 4200), coordinates(2527, 4264)],
                       [coordinates(2526, 850), coordinates(3354, 1083)],
                       [coordinates(2087, 4200), coordinates(2527, 4264)],
                       [coordinates(2526, 850), coordinates(3354, 1083)],
                       [coordinates(2087, 4200), coordinates(2527, 4264)],
                       [coordinates(2526, 850), coordinates(3354, 1083)],
                       [coordinates(2087, 4200), coordinates(2527, 4264)],
                       [coordinates(2526, 850), coordinates(3354, 1083)],
                       [coordinates(2087, 4200), coordinates(2527, 4264)],
                       [coordinates(2526, 850), coordinates(3354, 1083)],
                       [coordinates(2087, 4200), coordinates(2527, 4264)],
                       [coordinates(2526, 850), coordinates(3354, 1083)],
                       [coordinates(2087, 4200), coordinates(2527, 4264)],
                       [coordinates(2526, 850), coordinates(3354, 1083)],
                       [coordinates(2087, 4200), coordinates(2527, 4264)],
                       [coordinates(2526, 850), coordinates(3354, 1083)],
                       [coordinates(2087, 4200), coordinates(2527, 4264)],
                       [coordinates(2526, 850), coordinates(3354, 1083)]],
                       layer = 0)

sample.Merge(DE)
print(sample.coaxmons[0].R4)

resonator length:  4168.428279658478
resonator length:  4361.858826468799
resonator length:  4078.0068423988687
resonator length:  4107.708276096932
resonator length:  4137.845533630953
resonator length:  3855.0126197617187
resonator length:  4048.7318470888767
resonator length:  3908.4431481563156
resonator length:  3963.3755888052906
resonator length:  4199.466464942242
resonator length:  4019.874171740974
resonator length:  4328.38331740717
resonator length:  4395.856167316785
resonator length:  4430.387637636602
resonator length:  4262.950462961637
resonator length:  4230.970339428009
resonator length:  3991.4249560587164
resonator length:  3935.7176991133742
resonator length:  4295.41771709031
resonator length:  3881.544021163255
<class 'list'> [(1127.0, 4364.0), (1127.0, 4200), (1180.0905240615855, 4264), (1180.0905240615855, 3882.577973999999)]
<class 'gdspy.PolygonSet'>
<class 'list'> [(1127.0, 700), (1127.0, 850), (1202.482, 1083), (1202.482, 1117.422)]
<class 'gdspy.PolygonSe

In [5]:
sample.Generate_Grid(10000,5000,4,5)
#gdspy.LayoutViewer(depth = 2)

In [6]:
print(freq)

[7.12105263e+09 6.80526316e+09 7.27894737e+09 7.22631579e+09
 7.17368421e+09 7.70000000e+09 7.33157895e+09 7.59473684e+09
 7.48947368e+09 7.06842105e+09 7.38421053e+09 6.85789474e+09
 6.75263158e+09 6.70000000e+09 6.96315790e+09 7.01578947e+09
 7.43684210e+09 7.54210526e+09 6.91052632e+09 7.64736842e+09]


In [7]:
gdspy.write_gds('coaxmons_test.gds', cells=None, name='library', unit=1e-06, precision=1e-09, timestamp=None, binary_cells=None)